In [1]:
import sys
import os
import urllib
import time

import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

from PIL import Image

from tf_trt_models.classification import download_classification_checkpoint, build_classification_graph

%matplotlib inline

W0902 22:20:02.754085 140279592744768 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0902 22:20:02.755649 140279592744768 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/slim-0.1-py3.6.egg/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0902 22:20:02.801876 140279592744768 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/slim-0.1-py3.6.egg/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [2]:
import ipywidgets as widgets

In [3]:
MODELS = ['mobilenet_v1_0p25_128', 'mobilenet_v1_0p5_160', 'mobilenet_v1_1p0_224', 'mobilenet_v2_1p0_224',
          'vgg_16', 'vgg_19', 'inception_v1',
          'inception_v2', 'inception_v3', 'inception_v4', 'inception_resnet_v2', 'resnet_v1_50', 'resnet_v1_101',
          'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152']

w = widgets.Dropdown(options=MODELS,
                 value='mobilenet_v1_0p25_128',
                 description='model:',
                 disabled=False,)
display(w)

Dropdown(description='model:', options=('mobilenet_v1_0p25_128', 'mobilenet_v1_0p5_160', 'mobilenet_v1_1p0_224…

In [4]:
MODEL = w.value
# NUM_CLASSES = 1001
NUM_CLASSES = 1000
LABELS_PATH = './data/imagenet_labels_%d.txt' % NUM_CLASSES
IMAGE_PATH = './data/dog-yawning.jpg'

### Download the checkpoint and sample image

In [5]:
checkpoint_path = download_classification_checkpoint(MODEL, 'data')

### Build the frozen graph

In [6]:
frozen_graph, input_names, output_names = build_classification_graph(
    model=MODEL,
    checkpoint=checkpoint_path,
    num_classes=NUM_CLASSES
)
print(input_names, output_names)

W0902 22:19:15.942493 139780808177472 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/tf_trt_models-0.0-py3.6.egg/tf_trt_models/classification.py:208: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0902 22:19:15.945118 139780808177472 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/tf_trt_models-0.0-py3.6.egg/tf_trt_models/classification.py:212: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0902 22:19:16.104775 139780808177472 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/tf_trt_models-0.0-py3.6.egg/tf_trt_models/classification.py:214: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 22:19:16.111153 139780808177472 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/slim-0.1-py3.6.egg/nets/resnet_v1.py:221: The name tf.variable_scope is deprecated. Please use tf.compat.v1.va

['input'] ['scores']


### Optimized the graph with TensorRT

In [7]:
converter = trt.TrtGraphConverter(
    input_graph_def=frozen_graph,
    nodes_blacklist=output_names, #output nodes
    max_batch_size=1,
    is_dynamic_op=True,
    max_workspace_size_bytes=trt.DEFAULT_TRT_MAX_WORKSPACE_SIZE_BYTES,
    precision_mode=trt.TrtPrecisionMode.FP16,
    minimum_segment_size=50)
trt_graph = converter.convert()

trt_engine_opts = len([1 for n in trt_graph.node if str(n.op) == 'TRTEngineOp'])
print("trt_engine_opts = {}".format(trt_engine_opts))

trt_engine_opts = 0


In [8]:
MODEL_DIR = os.path.join('.', 'model')
if tf.gfile.Exists(MODEL_DIR) == False:
    tf.gfile.MkDir(MODEL_DIR)

In [9]:
base_name = os.path.splitext(os.path.basename(checkpoint_path))[0]
save_model_file_name = base_name + '_frozen_fp16.pb'
with open(os.path.join(MODEL_DIR, save_model_file_name), 'wb') as f:
    f.write(trt_graph.SerializeToString())

### copy original model.

In [6]:
tf.reset_default_graph()
frozen_graph, input_names, output_names = build_classification_graph(
    model=MODEL,
    checkpoint=checkpoint_path,
    num_classes=NUM_CLASSES,
    is_remove_relu6=False
)

W0902 22:20:15.860859 140279592744768 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/tf_trt_models-0.0-py3.6.egg/tf_trt_models/classification.py:208: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0902 22:20:15.862471 140279592744768 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/tf_trt_models-0.0-py3.6.egg/tf_trt_models/classification.py:212: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0902 22:20:16.034950 140279592744768 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/tf_trt_models-0.0-py3.6.egg/tf_trt_models/classification.py:214: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 22:20:16.039225 140279592744768 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/slim-0.1-py3.6.egg/nets/resnet_v1.py:221: The name tf.variable_scope is deprecated. Please use tf.compat.v1.va

In [9]:
base_name = os.path.splitext(os.path.basename(checkpoint_path))[0]
save_model_file_name = base_name + '_frozen.pb'
with open(os.path.join(MODEL_DIR, save_model_file_name), 'wb') as f:
    f.write(frozen_graph.SerializeToString())

In [10]:
!ls ./model

mobilenet_v1_1.0_224_frozen.pb	     resnet_v1_50_frozen.pb
mobilenet_v1_1.0_224_frozen_fp16.pb  resnet_v1_50_frozen_fp16.pb
mobilenet_v2_1.0_224_frozen.pb	     vgg_16_frozen.pb
mobilenet_v2_1.0_224_frozen_fp16.pb  vgg_16_frozen_fp16.pb
